Our chatbot typically is designed to facilitate a conversational interface, particulary for assisting student or prospective students interacting with a bot representing George Mason University. 
Mr.Bot aims to respond appropriately to various user inputs, guiding them through common queries related to admissions, courses status inquiries and more.

1. User's input triggers different responses from the bot based on predefined patterns
2. The dictionary consists of regular expressions as keys and lists of corresponding responses as values.

key is a regular expression which is used for word spotting. Value is the possible answer.
If a match has been found by using the regular expression, Eliza Mr.Bot will randomly choose an answer from the dictionary values.


The regular expressions used are:

1. -(Hello|Hi|Hey) : Matches various forms of greetings from the user.
2. -university: Matches user mentions of the term "university".
3. -student : Matches queries related to being a student or applying for degrees.
4. -(study|course|classes|subject): Matches user queries related to studies.
5. -(admission|query|status|documents): Matches queries related to admissions or status.
6. -graduate: Matches queries related to graduate studies.
7. -undergraduate: Matches queries related to undergraduate studies.
8. -(international|citizen) : Matches queries related to international students.
9. -want : Matches user expressions of desires or intentions.
10. -will : Matches user expressions of plans or intentions.
11. -yes/no: Matches user affirmations.
12. -(thanks|thank you) : Matches expressions of gratitude from the user.
13. others: Default regular expression that matches any input if none of the above patterns match.

References used: 
[1] Provided by Professor github.com/norib016/Developing_A_Chatbot_Python/blob/master/eliza.py 
[2] A blog: Building a simple chatbot https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImJkYzRlMTA5ODE1 by Parul Pandey, September 17, 2018.
'''

In [1]:
import nltk
#nltk.download("punkt")
from nltk.tokenize import word_tokenize
import re
import random
import time
from threading import Timer

In [2]:
"""For example:
[user]: I want to apply for a graduate Masters in George Mason University.
[Mr. Bot]:which intake would you like to applyfor?"""



response_dict = {
    r'.*(Hello|Hi|Hey).*':
    [
        "Hello! How may I help you?",
        "Hi! How's your day going?",
        "Hey! Welcome back"
    ],
    
    r'(.*)university(.*)':
    [
        "Hello! I'm Mr.Bot Welcome to George Mason University\n Mason is a public sea-grant research university in Fairfax county, Virginia near Washington, D.C."
        "The university operates four campuses in virginia, which is located in Fairfax, Arlington, Front royal, and Prince William counties."
        "Motto is** Freedom and Learning**."
        "I'm here to help with your studies. What can I provide information on?"
        "If you need additional resources, consider checking the university's official website."
    ],
    
    r'i student(.*)':
    [
        "Are you a mason student?",
        "Please give me you Student|Employee ID?",
        "What's wrong? How may I help you?",
        "Give me more details, I would like to help you.",
        "Would you like to apply for Bachelor's degree?",
        "Would you like to apply for a Master's degree?",
        "Would you like to apply for a PhD?",
        "Can I get you Mason ID.",
        "Which department would you like to apply for?",
        
    ],
    
    r'.*(study|course|classes|subject).*':
        [
         "Sure! What specific course or subject are you interested in?",
         "Tell me more about your studies. Are there any specific topics you'd like to discuss?",
        ],
    
    r'.*(admission|query|status|documents).*':
    [
        "How can I help you? tell me more.",
        "Sure! What specific query do you have.",
        "Are you looking  for any specific guidance ?"
        "What query do you have %s?",
        "Would you like you know your admission status?",
        "Do you need any help with your application?",
        "Do you want to know your tution fee status?"
        "Feel free to ask for any tips or guidance you might need."
    ],
    
    r'i graduate(.*)':
    [
        "Are you a current enrolled Mason graduate?",
        "would you like to apply for a graduate degree?",
        "which intake would like to apply for?",
        "which course are you interested in?",
    ],
    
    r'i undergraduate(.*)':
    [
        "Are you a current enrolled Mason undergraduate?",
        "would you like to apply for an bachelors degree?",
        "which intake would like to apply for?",
        "which course are you interested in?",
    ],
    
    r'.*(international|citizen).*':
    [
        "How can I assist you?",
        "Tell me more about your status?",
        "Would you like to study in George Mason University?"
        "you need to submit your GRE and IELTS|TOFEL , SAT testscores, and your Memo of previous studies."
    ],
    
     r'i want (.*)':
        [
                "How much does it mean to you if you got %s?",
                "Are you sure, why do you want %s?"
        ],
    
    r'i will (.*)':
        [
                "Why will you %s?",
                "When will you %s?",
                "Could please tell more details about how will you %s?"
        ],
    r'(.*)ready(.*)':
        [
                "Good to know! Tell me more."
        ],
   
     r'(.*)yes(.*)':
        [
             "You seem quite positive",
             "I understand. Tell me more.",
             "Are you sure?",
             "That is good to know?"
        ], 
                
   r'.*(thanks|thank you).*':
        [
         "You're welcome! If you have more questions, feel free to ask.",
         "No problem! Let me know if there's anything else I can help you with.",
         "You're welcome! If there's anything else on your mind, just let me know."
        ],
    
    r'(.*)':
         [
                  "May I know if your an experienced person in this field?",
                  "Please tell me how can I help you?",        
                  "I see, please tell me more.",
                  "Very interesting, please tell me more, because there are few prerequisites for each course."
                   "I'm here to help with any questions you have. Feel free to ask about your studies or anything else.",
                 "Tell me more about your studies or any concerns you might have.",
                 "Feel free to share more details. I'm here to assist you."
         ]


}


In [3]:
# it will be used to tranform from 2nd person to 1st person
pronoun_dict = {
    "I": "you",
    "me": "you",
    "my": "your",
    "mine": "yours",
    "am": "are",
    "myself": "yourself",
    "I'm": "you're",
    "I've": "you've",
    "I'll": "you'll",
    "you": "I",
    "your": "my",
    "yours": "mine",
    "you're": "I'm",
    "you've": "I've",
    "you'll": "I'll",
    "yourself": "myself",
}

In [ ]:
import re
import random

def transform_pronouns(input_text):
    words = input_text.split()
    transformed_words = [pronoun_dict.get(word, word) for word in words]
    return ' '.join(transformed_words)

def get_response_from_file(sentence): # if there is not response match from the dictionary it gets from the file or else it will directly give response from the dictionary 
    try:
        file = open("C:\\Users\\ashri\\Documents\\AIT-526\\Graduatephd.txt", 'r',encoding = 'utf-8')
        lines = file.read()
        for pattern, responses in response_dict.items():     #this for loop initiates over each key-value pair in the dictionary
            if re.match(pattern, user_input, re.IGNORECASE): #checks if user input matches the current pattern(start of the string user input)
                response = random.choice(responses)
        for line in lines:
            pattern, response = line.strip().split(':')
            if re.search(pattern, user_input, re.IGNORECASE):
                return response.strip()
    except FileNotFoundError:
        return "I'm sorry, I don't have information on that."

    
def get_response(user_input):   #this function takes user input as its parameter.
    for pattern, responses in response_dict.items():     #this for loop initiates over each key-value pair in the dictionary
        if re.match(pattern, user_input, re.IGNORECASE): #checks if user input matches the current pattern(start of the string user input)
            response = random.choice(responses)          # if there is match it selects the randon=m response from the list
            if '%' in response:                          #indidcates a place holder for every repsonse.
                # If there is placeholder then it replaces with a appropriate value
                response = response % transform_pronouns(re.match(pattern, user_input, re.IGNORECASE).group(1))
            return response   
        # function returns the response and exits 
    #if not found in response dictionary check in the file
    file_response = get_response_from_file()
    if file_response:
        return file_resposne
    
    return response_dict['(.*)'][0]   # if there is no match it returns a default reponse from the dictionary

# Firslty, Mr.Bot greets user by asking their name
# Later, an infinite loop is wriiten such that Bot continues interacting with the user until a condiiton inside the loop is met
#If user types "quit", it exits the conversation by saying Goodbye message and breaks out of the loop.
def chatbot():
    print("Hello! I'm your Eliza Mr.Bot. What's your name?")
    user_name = input("You: ")

    while True:
        user_input = input(f"{user_name}: ")
        if user_input.lower() in ['quit', 'bye', 'exit']:
            print("Chatbot: Bye! Have a nice day.")
            break
        
    
        
#Before processing the input, punctuations are removed using regex and then tokenize the input by splitting it into words
        # Strip punctuation and tokenize
        user_input = re.sub(r'[^\w\s]', '', user_input)
        tokens = user_input.split()

        # Respond based on predefined outputs
        response = get_response(user_input)
        print("Mr.bot:", response)
        

if __name__ == "__main__":
    chatbot()


Hello! I'm your Eliza Mr.Bot. What's your name?
You: Ashu
Ashu: I would like to study at George Mason University
Mr.bot: Hello! I'm Mr.Bot Welcome to George Mason University
 Mason is a public sea-grant research university in Fairfax county, Virginia near Washington, D.C.The university operates four campuses in virginia, which is located in Fairfax, Arlington, Front royal, and Prince William counties.Motto is** Freedom and Learning**.I'm here to help with your studies. What can I provide information on?If you need additional resources, consider checking the university's official website.
Ashu: Alright!
Mr.bot: Please tell me how can I help you?
Ashu: I will go for a Masters degree
Mr.bot: When will you go for a Masters degree?
Ashu: I have already completed my undergraduation
Mr.bot: Good to know! Tell me more.
Ashu: I want to study Data Analytics
Mr.bot: Tell me more about your studies. Are there any specific topics you'd like to discuss?
Ashu: Is there any specific course on Big data